<a href="https://colab.research.google.com/github/kumuds4/BCH/blob/master/Making_the_Most_of_your_Colab_Subscription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making the Most of your Colab Subscription



## Faster GPUs

Users who have purchased one of Colab's paid plans have access to faster GPUs and more memory. You can upgrade your notebook's GPU settings in `Runtime > Change runtime type` in the menu to select from several accelerator options, subject to availability.

The free of charge version of Colab grants access to Nvidia's T4 GPUs subject to quota restrictions and availability.

You can see what GPU you've been assigned at any time by executing the following cell. If the execution result of running the code cell below is "Not connected to a GPU", you can change the runtime by going to `Runtime > Change runtime type` in the menu to enable a GPU accelerator, and then re-execute the code cell.


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In order to use a GPU with your notebook, select the `Runtime > Change runtime type` menu, and then set the hardware accelerator to the desired option.

## More memory

Users who have purchased one of Colab's paid plans have access to high-memory VMs when they are available. More powerful GPUs are always offered with high-memory VMs.



You can see how much memory you have available at any time by running the following code cell. If the execution result of running the code cell below is "Not using a high-RAM runtime", then you can enable a high-RAM runtime via `Runtime > Change runtime type` in the menu. Then select High-RAM in the Runtime shape toggle button. After, re-execute the code cell.


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

## Longer runtimes

All Colab runtimes are reset after some period of time (which is faster if the runtime isn't executing code). Colab Pro and Pro+ users have access to longer runtimes than those who use Colab free of charge.

## Background execution

Colab Pro+ users have access to background execution, where notebooks will continue executing even after you've closed a browser tab. This is always enabled in Pro+ runtimes as long as you have compute units available.



## Relaxing resource limits in Colab Pro

Your resources are not unlimited in Colab. To make the most of Colab, avoid using resources when you don't need them. For example, only use a GPU when required and close Colab tabs when finished.



If you encounter limitations, you can relax those limitations by purchasing more compute units via Pay As You Go. Anyone can purchase compute units via [Pay As You Go](https://colab.research.google.com/signup); no subscription is required.

## Send us feedback!

If you have any feedback for us, please let us know. The best way to send feedback is by using the Help > 'Send feedback...' menu. If you encounter usage limits in Colab Pro consider subscribing to Pro+.

If you encounter errors or other issues with billing (payments) for Colab Pro, Pro+, or Pay As You Go, please email [colab-billing@google.com](mailto:colab-billing@google.com).

## More Resources

### Working with Notebooks in Colab
- [Overview of Colab](/notebooks/basic_features_overview.ipynb)
- [Guide to Markdown](/notebooks/markdown_guide.ipynb)
- [Importing libraries and installing dependencies](/notebooks/snippets/importing_libraries.ipynb)
- [Saving and loading notebooks in GitHub](https://colab.research.google.com/github/googlecolab/colabtools/blob/main/notebooks/colab-github-demo.ipynb)
- [Interactive forms](/notebooks/forms.ipynb)
- [Interactive widgets](/notebooks/widgets.ipynb)

<a name="working-with-data"></a>
### Working with Data
- [Loading data: Drive, Sheets, and Google Cloud Storage](/notebooks/io.ipynb)
- [Charts: visualizing data](/notebooks/charts.ipynb)
- [Getting started with BigQuery](/notebooks/bigquery.ipynb)

### Machine Learning Crash Course
These are a few of the notebooks from Google's online Machine Learning course. See the [full course website](https://developers.google.com/machine-learning/crash-course/) for more.
- [Intro to Pandas DataFrame](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/pandas_dataframe_ultraquick_tutorial.ipynb)
- [Linear regression with tf.keras using synthetic data](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/linear_regression_with_synthetic_data.ipynb)


<a name="using-accelerated-hardware"></a>
### Using Accelerated Hardware
- [TensorFlow with GPUs](/notebooks/gpu.ipynb)
- [TPUs in Colab](/notebooks/tpu.ipynb)

<a name="machine-learning-examples"></a>

## Machine Learning Examples

To see end-to-end examples of the interactive machine learning analyses that Colab makes possible, check out these tutorials using models from [TensorFlow Hub](https://tfhub.dev).

A few featured examples:

- [Retraining an Image Classifier](https://tensorflow.org/hub/tutorials/tf2_image_retraining): Build a Keras model on top of a pre-trained image classifier to distinguish flowers.
- [Text Classification](https://tensorflow.org/hub/tutorials/tf2_text_classification): Classify IMDB movie reviews as either *positive* or *negative*.
- [Style Transfer](https://tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization): Use deep learning to transfer style between images.
- [Multilingual Universal Sentence Encoder Q&A](https://tensorflow.org/hub/tutorials/retrieval_with_tf_hub_universal_encoder_qa): Use a machine learning model to answer questions from the SQuAD dataset.
- [Video Interpolation](https://tensorflow.org/hub/tutorials/tweening_conv3d): Predict what happened in a video between the first and the last frame.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
# Essential Scientific and Machine Learning Libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, GRU, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

# Machine Learning and Statistical Libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

# Additional Utilities
import logging
import traceback

# Global Constants for Polar Code Simulation
BLOCK_LENGTH = 128  # N: Total block length
INFO_LENGTH = 64    # K: Information bit length
SNR_RANGE = np.linspace(-2, 6, 31)  # Signal-to-Noise Ratio range
NUM_TRIALS = 50   # Number of simulation trials
NUM_SAMPLES = 300  # Number of samples for machine learning
EPOCHS = 10        # Training epochs
BATCH_SIZE = 64     # Batch size for training

# Logging Configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s: %(message)s'
)





class CRC:
    """
    Cyclic Redundancy Check (CRC) Implementation
    """
    def __init__(self, poly_degree=16):
        """
        Initialize CRC with specific polynomial

        Args:
            poly_degree (int): Degree of CRC polynomial
        """
        # Common CRC polynomials
        self.polynomials = {
            16: 0x1021,   # CRC-16-CCITT
            32: 0x04C11DB7  # CRC-32
        }

        self.poly = self.polynomials.get(poly_degree, 0x1021)
        self.poly_degree = poly_degree

    def generate(self, data):
        """
        Generate CRC checksum

        Args:
            data (np.ndarray): Input data bits

        Returns:
            np.ndarray: CRC checksum bits
        """
        try:
            # Convert data to integer
            data_int = int(''.join(map(str, data)), 2)

            # Perform CRC calculation
            crc = self._calculate_crc(data_int)

            # Convert CRC to binary array
            crc_bits = np.array(list(map(int, f'{crc:0{self.poly_degree}b}')))

            return crc_bits

        except Exception as e:
            logging.error(f"CRC Generation Error: {e}")
            return None

    def _calculate_crc(self, data):
        """
        Internal CRC calculation method

        Args:
            data (int): Input data as integer

        Returns:
            int: Calculated CRC value
        """
        # Simplified CRC calculation
        crc = data
        for _ in range(self.poly_degree):
            if crc & (1 << (self.poly_degree - 1)):
                crc = (crc << 1) ^ self.poly
            else:
                crc <<= 1

        return crc & ((1 << self.poly_degree) - 1)

    def validate(self, data, checksum):
        """
        Validate data using CRC checksum

        Args:
            data (np.ndarray): Input data bits
            checksum (np.ndarray): CRC checksum bits

        Returns:
            bool: True if CRC is valid, False otherwise
        """
        try:
            # Recalculate CRC
            calculated_crc = self.generate(data)

            # Compare calculated CRC with received CRC
            return np.array_equal(calculated_crc, checksum)

        except Exception as e:
            logging.error(f"CRC Validation Error: {e}")
            return False

class PolarCodeGenerator:
    """
    Polar Code Generator with Advanced Channel Polarization
    """
    def __init__(self, N, K, design_snr=0):
        """
        Initialize Polar Code Generator

        Args:
            N (int): Total block length
            K (int): Number of information bits
            design_snr (float): Design Signal-to-Noise Ratio
        """
        self.N = N
        self.K = K
        self.design_snr = design_snr

        # Calculate code rate
        self.rate = K / N

        # Channel polarization
        self.channel_capacities = self._compute_channel_capacities()
        self.info_bit_positions = self._select_info_bit_positions()

    def _compute_channel_capacities(self):
        """
        Compute channel capacities using rate-aware Bhattacharyya parameter

        Returns:
            np.ndarray: Channel capacities
        """
        def bhattacharyya_parameter(design_snr, rate):
            """
            Calculate Bhattacharyya parameter with rate consideration

            Args:
                design_snr (float): Design Signal-to-Noise Ratio
                rate (float): Code rate R = K/N

            Returns:
                float: Adjusted Bhattacharyya parameter
            """
            # Convert SNR to linear scale
            snr_linear = 10 ** (design_snr / 10)

            # Rate-adjusted channel parameter
            adjusted_snr = snr_linear * rate

            # Compute Bhattacharyya parameter
            return np.exp(-adjusted_snr / 2)

        # Initialize channel capacities
        capacities = np.zeros(self.N)

        # Use rate-adjusted Bhattacharyya parameter
        z = bhattacharyya_parameter(self.design_snr, self.rate)

        capacities[0] = z
        for i in range(1, self.N):
            j = i // 2
            capacities[i] = (capacities[j] ** 2 if i % 2 == 0
                             else 1 - (1 - capacities[j] ** 2) ** 2)

        return capacities

    def _select_info_bit_positions(self):
        """
        Select best bit positions for information transmission

        Returns:
            np.ndarray: Indices of selected information bit positions
        """
        # Sort channel capacities and select best positions
        sorted_indices = np.argsort(self.channel_capacities)
        return np.sort(sorted_indices[-self.K:])

def encode(self, info_bits):
    """
    Advanced Polar Code Encoding with Comprehensive Diagnostics

    Args:
        info_bits (np.ndarray): Information bits to encode

    Returns:
        np.ndarray: Encoded codeword
    """
    try:
        # Convert input to numpy array if not already
        info_bits = np.asarray(info_bits)

        # Extensive input validation
        print("Encoding Diagnostics:")
        print(f"Input Type: {type(info_bits)}")
        print(f"Input Shape: {info_bits.shape}")
        print(f"Input Content: {info_bits}")
        print(f"Expected Info Length (K): {self.K}")

        # Validate input length
        if len(info_bits) != self.K:
            raise ValueError(f"Incorrect input length. Expected {self.K} bits, got {len(info_bits)}")

        # Validate binary input
        if not np.all(np.isin(info_bits, [0, 1])):
            raise ValueError(f"Non-binary bits detected: {info_bits}")

        # Create codeword with explicit type and initialization
        codeword = np.zeros(self.N, dtype=int)

        # Validate info bit positions
        print(f"Info Bit Positions: {self.info_bit_positions}")

        # Defensive position assignment
        try:
            codeword[self.info_bit_positions] = info_bits
        except Exception as assign_error:
            print(f"Position Assignment Error: {assign_error}")
            print(f"Codeword Length: {len(codeword)}")
            print(f"Max Position Index: {max(self.info_bit_positions)}")
            raise

        # Recursive encoding with error tracking
        try:
            for i in range(int(np.log2(self.N))):
                step = 2 ** i
                for j in range(0, self.N, 2 * step):
                    for k in range(j, j + step):
                        codeword[k + step] ^= codeword[k]
        except Exception as encode_error:
            print(f"Encoding Transformation Error: {encode_error}")
            raise

        # Final validation
        print(f"Final Codeword: {codeword}")
        print(f"Codeword Shape: {codeword.shape}")
        print(f"Codeword Length: {len(codeword)}")

        return codeword

    except Exception as e:
        print(f"Comprehensive Encoding Error: {e}")
        import traceback
        traceback.print_exc()
        raise



def simulate_channel(signal, snr, channel_type='AWGN'):
    """
    Advanced Channel Simulation with Complex Noise Modeling

    Args:
        signal (np.ndarray): Input signal
        snr (float): Signal-to-Noise Ratio in dB
        channel_type (str): Channel type (AWGN or Rayleigh)

    Returns:
        np.ndarray: Received noisy signal
    """
    try:
        # Convert SNR to linear scale
        snr_linear = 10 ** (snr / 10)

        # Noise power calculation with nuanced approach
        noise_std = np.sqrt(1 / (2 * snr_linear))

        if channel_type == 'AWGN':
            # Advanced Gaussian noise generation
            noise = np.random.normal(0, noise_std, signal.shape)

            # Non-linear noise characteristics
            noise += 0.1 * noise * np.sin(signal)  # Frequency-dependent noise
            noise *= np.random.uniform(0.8, 1.2, noise.shape)  # Amplitude variation

            received_signal = signal + noise

        elif channel_type == 'Rayleigh':
            # Advanced Rayleigh fading with complex model
            fading_coefficient = np.random.rayleigh(
                scale=1/np.sqrt(snr_linear),
                size=signal.shape
            )

            # Phase randomization
            phase_shift = np.random.uniform(0, 2*np.pi, signal.shape)

            # Multiplicative fading
            faded_signal = signal * fading_coefficient * np.exp(1j * phase_shift)

            # Add Gaussian noise
            noise = np.random.normal(0, noise_std, signal.shape)

            received_signal = np.real(faded_signal) + noise

        else:
            raise ValueError(f"Unsupported channel type: {channel_type}")

        return received_signal

    except Exception as e:
        logging.error(f"Channel Simulation Error: {e}")
        traceback.print_exc()
        return None

def generate_random_noise(signal, noise_type='gaussian'):
    """
    Generate advanced random noise for signal augmentation

    Args:
        signal (np.ndarray): Input signal
        noise_type (str): Type of noise generation

    Returns:
        np.ndarray: Noise-augmented signal
    """
    try:
        if noise_type == 'gaussian':
            # Standard Gaussian noise
            noise = np.random.normal(0, 0.1, signal.shape)

        elif noise_type == 'uniform':
            # Uniform noise
            noise = np.random.uniform(-0.1, 0.1, signal.shape)

        elif noise_type == 'non_linear':
            # Non-linear noise with multiple components
            noise = (
                np.random.normal(0, 0.05, signal.shape) *
                np.sin(signal) *
                np.random.uniform(0.8, 1.2, signal.shape)
            )

        else:
            raise ValueError(f"Unsupported noise type: {noise_type}")

        return signal + noise

    except Exception as e:
        logging.error(f"Noise Generation Error: {e}")
        traceback.print_exc()
class SuccessiveCancellationListDecoder:
    """
    Successive Cancellation List (SCL) Decoder with Random Codeword Assistance
    """
    def __init__(self, N, K, list_size=8, CRC_length=16):
        """
        Initialize SCL Decoder

        Args:
            N (int): Block length
            K (int): Information length
            list_size (int): Number of candidate paths
            CRC_length (int): CRC polynomial length
        """
        self.N = N
        self.K = K
        self.list_size = list_size
        self.crc = CRC(poly_degree=CRC_length)

        # Polarization parameters
        self.polar_code_gen = PolarCodeGenerator(N=N, K=K)
        self.channel_capacities = self.polar_code_gen.channel_capacities
        self.info_bit_positions = self._select_info_bit_positions()

    def _select_info_bit_positions(self):
        """
        Select best bit positions for information transmission

        Returns:
            np.ndarray: Indices of selected information bit positions
        """
        sorted_indices = np.argsort(self.channel_capacities)
        return np.sort(sorted_indices[-self.K:])

    def generate_random_codewords(self, num_codewords):
        """
        Generate Random Codewords

        Args:
            num_codewords (int): Number of random codewords to generate

        Returns:
            tuple: Random information bits and encoded codewords
        """
        # Generate random information bits
        random_info_bits = np.random.randint(2, size=(num_codewords, self.K))

        # Encode random codewords
        random_codewords = np.array([
            self.polar_code_gen.encode(bits) for bits in random_info_bits
        ])

        return random_info_bits, random_codewords

    def scl_decode(self, received_signal, random_codewords=None):
        """
        Successive Cancellation List Decoding with Random Codeword Assistance

        Args:
            received_signal (np.ndarray): Received noisy signal
            random_codewords (np.ndarray, optional): Pre-generated random codewords

        Returns:
            np.ndarray: Decoded information bits
        """
        # Generate random codewords if not provided
        if random_codewords is None:
            _, random_codewords = self.generate_random_codewords(num_codewords=100)

        # Initialize paths
        paths = [{'bits': np.zeros(self.N, dtype=int),
                  'metric': 0.0,
                  'path_metric': 0.0} for _ in range(self.list_size)]

        # Recursive decoding
        for bit_position in range(self.N):
            # Expand paths
            new_paths = []
            for path in paths:
                # Try both 0 and 1 for current bit
                for bit_value in [0, 1]:
                    new_path = path.copy()
                    new_path['bits'][bit_position] = bit_value

                    # Update path metric with random codeword assistance
                    path_metric = self._compute_path_metric(
                        new_path['bits'],
                        received_signal,
                        random_codewords
                    )
                    new_path['path_metric'] = path_metric
                    new_paths.append(new_path)

            # Sort and prune paths
            new_paths.sort(key=lambda x: x['path_metric'])
            paths = new_paths[:self.list_size]

        # Select best path with CRC validation
        for path in paths:
            # Extract info bits
            info_bits = path['bits'][self.info_bit_positions]

            # Generate and validate CRC
            crc_bits = self.crc.generate(info_bits)
            if self.crc.validate(info_bits, crc_bits):
                return info_bits

        # Fallback to first path if no CRC-valid path found
        return paths[0]['bits'][self.info_bit_positions]

    def _compute_path_metric(self, path_bits, received_signal, random_codewords):
        """
        Compute path metric using log-likelihood with random codeword assistance

        Args:
            path_bits (np.ndarray): Candidate path bits
            received_signal (np.ndarray): Received noisy signal
            random_codewords (np.ndarray): Pre-generated random codewords

        Returns:
            float: Path metric
        """
        # Compute basic log-likelihood
        base_metric = -np.sum(np.abs(path_bits - received_signal))

        # Random codeword assistance
        random_metrics = [
            -np.sum(np.abs(path_bits - codeword))
            for codeword in random_codewords
        ]

        # Combine metrics
        combined_metric = base_metric + np.mean(random_metrics)

        return combined_metric

class RNNDecoder:
    """
    Recurrent Neural Network (RNN) Decoder with Random Codeword Augmentation
    """
    def __init__(self, input_shape, num_classes=2):
        """
        Initialize RNN Decoder

        Args:
            input_shape (tuple): Input shape for the RNN
            num_classes (int): Number of output classes
        """
        self.model = self._build_model(input_shape, num_classes)

    def _build_model(self, input_shape, num_classes):
        """
        Build RNN Model Architecture with Random Codeword Features

        Args:
            input_shape (tuple): Input shape
            num_classes (int): Number of output classes

        Returns:
            tf.keras.Model: Compiled RNN model
        """
        model = Sequential([
            Input(shape=input_shape),
            LSTM(64, return_sequences=True, kernel_regularizer=l2(0.001)),
            Dropout(0.3),
            GRU(32, kernel_regularizer=l2(0.001)),
            Dropout(0.2),
            Dense(16, activation='relu', kernel_regularizer=l2(0.001)),
            Dense(num_classes, activation='softmax')
        ])

        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        return model

    def train(self, X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE):
        """
        Train RNN Decoder with Random Codeword Augmentation

        Args:
            X_train (np.ndarray): Training features
            y_train (np.ndarray): Training labels
            epochs (int): Number of training epochs
            batch_size (int): Batch size for training

        Returns:
            Training history
        """
        # Ensure input is 3D for RNN
        if len(X_train.shape) == 2:
            X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

        # One-hot encode labels
        y_train_onehot = tf.keras.utils.to_categorical(y_train)

        # Early stopping and learning rate reduction
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=20,
            restore_best_weights=True
        )

        reduce_lr = ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=10,
            min_lr=1e-5
        )

        # Train the model
        history = self.model.fit(
            X_train,
            y_train_onehot,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=0.2,
            callbacks=[early_stopping, reduce_lr],
            verbose=1
        )

        return history

def generate_performance_metrics(block_length, info_length, snr_range):
    """
    Generate Comprehensive Performance Metrics

    Args:
        block_length (int): Total block length
        info_length (int): Number of information bits
        snr_range (np.ndarray): Signal-to-Noise Ratio range

    Returns:
        tuple: SNR range, BER values, BLER values
    """
    try:
        # Initialize Polar Code Generator
        polar_code_gen = PolarCodeGenerator(N=block_length, K=info_length)

        # Initialize SCL Decoder
        scl_decoder = SuccessiveCancellationListDecoder(
            N=block_length,
            K=info_length
        )

        # Generate random codewords for decoder assistance
        _, random_codewords = scl_decoder.generate_random_codewords(num_codewords=300)

        # Initialize result storage
        ber_values = np.zeros_like(snr_range, dtype=float)
        bler_values = np.zeros_like(snr_range, dtype=float)

        # Iterate through SNR points
        for idx, snr in enumerate(snr_range):
            total_bit_errors = 0
            total_block_errors = 0

            # Multiple trials for statistical significance
            for _ in range(NUM_TRIALS):
                # Generate random information bits
                info_bits = np.random.randint(2, size=info_length)

                # Encode
                encoded_signal = polar_code_gen.encode(info_bits)

                # Multiple channel and noise realizations
                trial_bit_errors = 0

                for _ in range(10):  # Increased noise realizations
                    # Simulate channel
                    received_signal = simulate_channel(
                        encoded_signal,
                        snr,
                        channel_type='AWGN'
                    )

                    # SCL Decoding with random codeword assistance
                    decoded_bits = scl_decoder.scl_decode(
                        received_signal,
                        random_codewords
                    )

                    # Error calculation
                    bit_errors = np.sum(np.abs(decoded_bits - info_bits))
                    trial_bit_errors += bit_errors

                # Average errors across channel realizations
                avg_trial_bit_errors = trial_bit_errors / 10

                total_bit_errors += avg_trial_bit_errors
                total_block_errors += (avg_trial_bit_errors > 0)

            # Compute average error rates
            ber_values[idx] = total_bit_errors / (NUM_TRIALS * info_length)
            bler_values[idx] = total_block_errors / NUM_TRIALS

            # Logging
            print(f"SNR: {snr} dB, BER: {ber_values[idx]:.4e}, BLER: {bler_values[idx]:.4e}")

        return snr_range, ber_values, bler_values

    except Exception as e:
        logging.error(f"Performance Metrics Error: {e}")
        traceback.print_exc()
        return None, None, None

def generate_scl_performance_metrics(block_length, info_length, snr_range,
                                      list_sizes=[1, 8, 16],
                                      channel_type='AWGN'):
    """
    Generate SCL Performance Metrics for Different List Sizes

    Args:
        block_length (int): Total block length
        info_length (int): Information bit length
        snr_range (np.ndarray): Signal-to-Noise Ratio range
        list_sizes (list): List sizes for SCL decoding
        channel_type (str): Channel type (AWGN or Rayleigh)

    Returns:
        dict: Performance metrics for different list sizes
    """
    try:
        # Initialize Polar Code Generator
        polar_code_gen = PolarCodeGenerator(N=block_length, K=info_length)

        scl_performance_metrics = {}

        for list_size in list_sizes:
            # Initialize SCL Decoder
            scl_decoder = SuccessiveCancellationListDecoder(
                N=block_length,
                K=info_length,
                list_size=list_size
            )

            # Generate random codewords
            _, random_codewords = scl_decoder.generate_random_codewords(num_codewords=50)

            # Initialize error tracking
            ber_values = np.zeros_like(snr_range, dtype=float)
            bler_values = np.zeros_like(snr_range, dtype=float)

            # Iterate through SNR points
            for snr_idx, snr in enumerate(snr_range):
                total_bit_errors = 0
                total_block_errors = 0

                # Multiple trials
                for _ in range(NUM_TRIALS):
                    # Generate random information bits
                    info_bits = np.random.randint(2, size=info_length)

                    # Encode
                    encoded_signal = polar_code_gen.encode(info_bits)

                    # Simulate channel
                    received_signal = simulate_channel(
                        encoded_signal,
                        snr,
                        channel_type=channel_type
                    )

                    # SCL Decoding
                    decoded_bits = scl_decoder.scl_decode(
                        received_signal,
                        random_codewords
                    )

                    # Calculate errors
                    bit_errors = np.sum(np.abs(decoded_bits - info_bits))
                    total_bit_errors += bit_errors
                    total_block_errors += (bit_errors > 0)

                # Compute average error rates
                ber_values[snr_idx] = total_bit_errors / (NUM_TRIALS * info_length)
                bler_values[snr_idx] = total_block_errors / NUM_TRIALS

            # Store performance for this list size
            scl_performance_metrics[list_size] = {
                'ber': ber_values,
                'bler': bler_values
            }

        return scl_performance_metrics

    except Exception as e:
        logging.error(f"SCL Performance Metrics Error: {e}")
        traceback.print_exc()
        return None

def plot_detailed_performance(snr_range,
                               scl_performance_metrics_awgn,
                               scl_performance_metrics_rayleigh):
    """
    Plot Detailed SCL Performance for AWGN and Rayleigh Channels

    Args:
        snr_range (np.ndarray): Signal-to-Noise Ratio range
        scl_performance_metrics_awgn (dict): AWGN channel performance metrics
        scl_performance_metrics_rayleigh (dict): Rayleigh channel performance metrics
    """
    plt.figure(figsize=(20, 10))

    # BER Plot
    plt.subplot(1, 2, 1)
    plt.title('Bit Error Rate - SCL Decoding', fontsize=14)

    # AWGN Channel BER
    for list_size, metrics in scl_performance_metrics_awgn.items():
        plt.semilogy(snr_range, metrics['ber'],
                     label=f'AWGN List Size = {list_size}',
                     marker='o')

    # Rayleigh Channel BER
    for list_size, metrics in scl_performance_metrics_rayleigh.items():
        plt.semilogy(snr_range, metrics['ber'],
                     label=f'Rayleigh List Size = {list_size}',
                     marker='s')

    plt.xlabel('SNR (dB)', fontsize=12)
    plt.ylabel('Bit Error Rate', fontsize=12)
    plt.ylim(1e-5, 1e0)
    plt.grid(True, which='both', ls='-', alpha=0.5)
    plt.legend(fontsize=10)

    # BLER Plot
    plt.subplot(1, 2, 2)
    plt.title('Block Error Rate - SCL Decoding', fontsize=14)

    # AWGN Channel BLER
    for list_size, metrics in scl_performance_metrics_awgn.items():
        plt.semilogy(snr_range, metrics['bler'],
                     label=f'AWGN List Size = {list_size}',
                     marker='o')

    # Rayleigh Channel BLER
    for list_size, metrics in scl_performance_metrics_rayleigh.items():
        plt.semilogy(snr_range, metrics['bler'],
                     label=f'Rayleigh List Size = {list_size}',
                     marker='s')

    plt.xlabel('SNR (dB)', fontsize=12)
    plt.ylabel('Block Error Rate', fontsize=12)
    plt.ylim(1e-5, 1e0)
    plt.grid(True, which='both', ls='-', alpha=0.5)
    plt.legend(fontsize=10)

    plt.tight_layout()
    plt.show()

def plot_training_performance(history, model_name):
    """
    Comprehensive Training Performance Visualization

    Args:
        history: Training history from Keras model
        model_name (str): Name of the model (e.g., 'RNN', 'Deep Learning')
    """
    plt.figure(figsize=(20, 10))

    # Accuracy Subplot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'{model_name} Model - Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    # Loss Subplot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'{model_name} Model - Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

def plot_confusion_matrices(models, X_test, y_test):
    """
    Plot Separate Confusion Matrices for Different Models

    Args:
        models (dict): Dictionary of trained models
        X_test (np.ndarray): Test features
        y_test (np.ndarray): Test labels
    """
    plt.figure(figsize=(15, 6))

    # ML Decoder Confusion Matrix
    plt.subplot(1, 2, 1)
    ml_predictions = models['ml'].predict(X_test)
    ml_pred_classes = ml_predictions['random_forest']
    cm_ml = confusion_matrix(y_test, ml_pred_classes)

    plt.title('ML Decoder - Confusion Matrix', fontsize=14)
    sns.heatmap(cm_ml, annot=True, fmt='d', cmap='Blues',
                cbar_kws={'label': 'Number of Samples'})
    plt.xlabel('Predicted Label', fontsize=12)
    plt.ylabel('True Label', fontsize=12)

    # RNN Decoder Confusion Matrix
    plt.subplot(1, 2, 2)
    rnn_predictions = models['rnn'].predict(X_test)
    rnn_pred_classes = np.argmax(rnn_predictions, axis=1)
    cm_rnn = confusion_matrix(y_test, rnn_pred_classes)

    plt.title('RNN Decoder - Confusion Matrix', fontsize=14)
    sns.heatmap(cm_rnn, annot=True, fmt='d', cmap='Greens',
                cbar_kws={'label': 'Number of Samples'})
    plt.xlabel('Predicted Label', fontsize=12)
    plt.ylabel('True Label', fontsize=12)

    plt.tight_layout()
    plt.show()


def prepare_ml_dataset_with_codewords(polar_code_gen, num_samples):
    try:
        # Extensive logging and validation
        print("Preparing ML Dataset:")
        print(f"Polar Code Generator: {polar_code_gen}")
        print(f"Number of Samples: {num_samples}")
        print(f"Block Length (N): {polar_code_gen.N}")
        print(f"Info Length (K): {polar_code_gen.K}")

        # Generate random information bits with type specification
        info_bits_list = np.random.randint(2, size=(num_samples, polar_code_gen.K), dtype=int)
        print(f"Info Bits Shape: {info_bits_list.shape}")

        # Defensive codeword generation with detailed tracking
        codewords = []
        encoding_errors = 0

        for idx, bits in enumerate(info_bits_list):
            try:
                # Detailed encoding attempt with comprehensive checks
                print(f"\nEncoding attempt {idx}:")
                print(f"Input Bits: {bits}")
                print(f"Input Bits Type: {type(bits)}")

                # Explicit type conversion and validation
                bits = np.asarray(bits, dtype=int)

                # Verify input bits
                if len(bits) != polar_code_gen.K:
                    print(f"Incorrect bit length. Expected {polar_code_gen.K}, got {len(bits)}")
                    encoding_errors += 1
                    continue

                # Ensure bits are binary
                if not np.all(np.isin(bits, [0, 1])):
                    print(f"Non-binary bits detected: {bits}")
                    encoding_errors += 1
                    continue

                # Attempt encoding with comprehensive error handling
                encoded = polar_code_gen.encode(bits)

                # Validate encoded signal
                if encoded is None:
                    print(f"Encoding returned None for bits {bits}")
                    encoding_errors += 1
                    continue

                print(f"Encoded Signal Shape: {encoded.shape}")
                print(f"Encoded Signal: {encoded}")

                codewords.append(encoded)

            except Exception as encode_error:
                print(f"Encoding Error for bits {bits}: {encode_error}")
                encoding_errors += 1
                continue


                # Explicit type conversion and validation
                bits = np.asarray(bits, dtype=int)

                # Verify input bits
                if len(bits) != polar_code_gen.K:
                    print(f"Incorrect bit length. Expected {polar_code_gen.K}, got {len(bits)}")
                    encoding_errors += 1
                    continue

                # Ensure bits are binary
                if not np.all(np.isin(bits, [0, 1])):
                    print(f"Non-binary bits detected: {bits}")
                    encoding_errors += 1
                    continue

                # Attempt encoding with comprehensive error handling
                encoded = polar_code_gen.encode(bits)

                # Validate encoded signal
                if encoded is None:
                    print(f"Encoding returned None for bits {bits}")
                    encoding_errors += 1
                    continue

                print(f"Encoded Signal Shape: {encoded.shape}")
                print(f"Encoded Signal: {encoded}")

                codewords.append(encoded)

            except Exception as encode_error:
                print(f"Encoding Error for bits {bits}: {encode_error}")
                encoding_errors += 1
                continue

        # Rest of the function remains the same...






        # Generate random information bits
        info_bits_list = np.random.randint(2, size=(num_samples, polar_code_gen.K))
        print(f"Info Bits Shape: {info_bits_list.shape}")

        # Defensive codeword generation with detailed tracking
        codewords = []
        encoding_errors = 0

        for idx, bits in enumerate(info_bits_list):
            try:
                # Detailed encoding attempt
                print(f"Encoding attempt {idx}:")
                print(f"Input Bits: {bits}")

                # Verify input bits
                if len(bits) != polar_code_gen.K:
                    print(f"Incorrect bit length. Expected {polar_code_gen.K}, got {len(bits)}")
                    continue

                # Ensure bits are binary
                if not np.all(np.isin(bits, [0, 1])):
                    print(f"Non-binary bits detected: {bits}")
                    continue

                # Attempt encoding
                encoded = polar_code_gen.encode(bits)

                # Validate encoded signal
                if encoded is None:
                    print(f"Encoding returned None for bits {bits}")
                    encoding_errors += 1
                    continue

                print(f"Encoded Signal Shape: {encoded.shape}")
                print(f"Encoded Signal: {encoded}")

                codewords.append(encoded)

            except Exception as encode_error:
                print(f"Encoding Error for bits {bits}: {encode_error}")
                encoding_errors += 1
                continue

        # Convert to numpy array
        codewords = np.array(codewords)

        print(f"Total Encoding Errors: {encoding_errors}")
        print(f"Codewords Shape: {codewords.shape}")

        # Validate codewords
        if len(codewords) == 0:
            # Detailed diagnostics if no codewords generated
            print("Diagnostic Information:")
            print(f"Original Info Bits Shape: {info_bits_list.shape}")
            print("Sample of Info Bits:")
            for i in range(min(5, len(info_bits_list))):
                print(f"Bits {i}: {info_bits_list[i]}")

            raise ValueError("No valid codewords generated. Check encoding process.")

        # Ensure consistent shape
        if codewords.ndim == 1:
            codewords = codewords.reshape(1, -1)

        # Truncate to num_samples if necessary
        codewords = codewords[:num_samples]
        info_bits_list = info_bits_list[:num_samples]

        # Create features with multiple characteristics
        X = np.zeros((len(codewords), 9))

        # Feature engineering with comprehensive error handling
        X[:, 0] = np.mean(codewords, axis=1)    # Mean of codeword
        X[:, 1] = np.std(codewords, axis=1)     # Standard deviation
        X[:, 2] = np.sum(codewords, axis=1)     # Sum of codeword
        X[:, 3] = np.max(codewords, axis=1)     # Max value
        X[:, 4] = np.min(codewords, axis=1)     # Min value
        X[:, 5] = np.count_nonzero(codewords, axis=1)  # Number of non-zero bits
        X[:, 6] = np.sum(info_bits_list, axis=1)  # Information bits sum
        X[:, 7] = np.var(codewords, axis=1)     # Variance
        X[:, 8] = np.median(codewords, axis=1)  # Median

        # Binary classification labels
        y = (X[:, 2] > np.median(X[:, 2])).astype(int)

        print(f"X Shape: {X.shape}")
        print(f"y Shape: {y.shape}")

        return X, y

    except Exception as e:
        print(f"Comprehensive Dataset Preparation Error: {e}")
        import traceback
        traceback.print_exc()
        return None, None


def main():
    try:


        # Prepare Machine Learning Dataset
        polar_code_gen = PolarCodeGenerator(N=BLOCK_LENGTH, K=INFO_LENGTH)
        X, y = prepare_ml_dataset_with_codewords(polar_code_gen, num_samples=NUM_SAMPLES)

         # Prepare Machine Learning Dataset with extensive logging
        print("Starting Dataset Preparation:")
        print(f"Block Length: {BLOCK_LENGTH}")
        print(f"Info Length: {INFO_LENGTH}")
        print(f"Number of Samples: {NUM_SAMPLES}")
        # Validate dataset
        if X is None or y is None:
            print("Failed to prepare dataset")
        # Prepare Machine Learning Dataset
        X, y = prepare_ml_dataset_with_codewords(
            PolarCodeGenerator(N=BLOCK_LENGTH, K=INFO_LENGTH),
            num_samples=NUM_SAMPLES
        )

        # Split Dataset
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42
        )

        # Train RNN Decoder
        rnn_decoder = RNNDecoder(input_shape=(X_train.shape[1], 1), num_classes=2)
        rnn_history = rnn_decoder.train(X_train, y_train)

        # Plot RNN Training Performance
        plot_training_performance(rnn_history, 'RNN')

        # Train ML Decoder
        ml_decoder = MLDecoder()
        ml_models = ml_decoder.train_models(X_train, y_train)

        # Prepare models for confusion matrix
        models = {
            'rnn': rnn_decoder.model,
            'ml': ml_decoder
        }

        # Plot Confusion Matrices
        plot_confusion_matrices(models, X_test, y_test)

        # Rest of the function remains the same...



        # Previous code for RNN, ML training and confusion matrices...

        # Comprehensive Performance Visualization
        plot_comprehensive_performance(
            snr_awgn, ber_awgn_traditional, bler_awgn_traditional,
            ber_awgn_ml, bler_awgn_ml,
            snr_rayleigh, ber_rayleigh_traditional, bler_rayleigh_traditional,
            ber_rayleigh_ml, bler_rayleigh_ml,
            ml_model=ml_models['random_forest'],
            rnn_model=rnn_decoder.model,
            X_test=X_test,
            y_test=y_test
        )

        # SCL Performance Analysis for AWGN Channel
        scl_performance_metrics_awgn = generate_scl_performance_metrics(
            block_length=BLOCK_LENGTH,
            info_length=INFO_LENGTH,
            snr_range=SNR_RANGE,
            list_sizes=[1, 8, 16],
            channel_type='AWGN'
        )

        # SCL Performance Analysis for Rayleigh Channel
        scl_performance_metrics_rayleigh = generate_scl_performance_metrics(
            block_length=BLOCK_LENGTH,
            info_length=INFO_LENGTH,
            snr_range=SNR_RANGE,
            list_sizes=[1, 8, 16],
            channel_type='Rayleigh'
        )

        # Plot SCL Performance
        plot_detailed_performance(
            SNR_RANGE,
            scl_performance_metrics_awgn,
            scl_performance_metrics_rayleigh
        )

        # Print Detailed Classification Reports
        print("\nRNN Decoder Classification Report:")
        rnn_predictions = rnn_decoder.predict(X_test.reshape(-1, X_test.shape[1], 1))
        rnn_pred_classes = np.argmax(rnn_predictions, axis=1)
        print(classification_report(y_test, rnn_pred_classes))

        print("\nML Decoder Classification Reports:")
        ml_predictions = ml_decoder.predict(X_test)
        for name, predictions in ml_predictions.items():
            print(f"\n{name.upper()} Classifier:")
            print(classification_report(y_test, predictions))

        # Optional: Feature Importance Visualization
        plt.figure(figsize=(10, 6))
        feature_names = [f'Feature {i+1}' for i in range(X_test.shape[1])]

        # Random Forest Feature Importance
        if 'random_forest' in ml_models:
            rf_model = ml_models['random_forest']
            plt.bar(feature_names, rf_model.feature_importances_)
            plt.title('Random Forest - Feature Importance')
            plt.xlabel('Features')
            plt.ylabel('Importance')
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.show()

    except Exception as e:
        print(f"Comprehensive Simulation Error: {e}")
        import traceback
        traceback.print_exc()

# Run the simulation
if __name__ == "__main__":
    main()


Streaming output truncated to the last 5000 lines.
Input Bits Type: <class 'numpy.ndarray'>
Encoding Error for bits [1 1 1 1 0 0 1 1 0 0 1 1 1 0 1 1 1 0 1 1 0 1 0 1 1 0 1 0 1 0 1 1 0 0 1 1 1
 1 1 0 0 1 0 1 1 1 1 0 0 1 1 0 0 0 1 1 0 1 1 0 0 0 0 0]: 'PolarCodeGenerator' object has no attribute 'encode'

Encoding attempt 220:
Input Bits: [1 0 1 0 0 0 0 1 1 0 1 0 1 0 0 0 0 1 1 0 1 0 1 0 1 1 0 1 1 1 1 0 1 0 1 0 1
 0 0 1 1 0 1 0 0 1 0 0 0 0 1 0 1 1 1 1 0 1 0 0 0 0 0 1]
Input Bits Type: <class 'numpy.ndarray'>
Encoding Error for bits [1 0 1 0 0 0 0 1 1 0 1 0 1 0 0 0 0 1 1 0 1 0 1 0 1 1 0 1 1 1 1 0 1 0 1 0 1
 0 0 1 1 0 1 0 0 1 0 0 0 0 1 0 1 1 1 1 0 1 0 0 0 0 0 1]: 'PolarCodeGenerator' object has no attribute 'encode'

Encoding attempt 221:
Input Bits: [0 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1 0 1 1 0 1 1 0 0 1 1 1 1
 1 0 1 1 1 0 1 1 1 0 0 0 0 0 1 0 0 1 1 0 0 1 1 1 1 1 0]
Input Bits Type: <class 'numpy.ndarray'>
Encoding Error for bits [0 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 1 1 0 1

Traceback (most recent call last):
  File "<ipython-input-21-78ec57bde087>", line 1053, in prepare_ml_dataset_with_codewords
    raise ValueError("No valid codewords generated. Check encoding process.")
ValueError: No valid codewords generated. Check encoding process.


Encoding Error for bits [0 1 1 0 0 0 1 1 0 0 0 1 0 0 0 0 1 1 1 0 1 1 0 1 1 0 1 0 0 0 1 1 1 0 1 0 1
 1 0 1 0 1 0 0 1 1 0 1 0 1 1 0 0 1 0 0 0 0 1 0 0 1 1 0]: 'PolarCodeGenerator' object has no attribute 'encode'
Encoding attempt 162:
Input Bits: [1 1 1 1 1 0 1 0 0 0 0 1 1 0 1 1 0 0 0 1 0 0 0 1 1 0 1 1 1 0 0 1 0 0 0 1 1
 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0]
Encoding Error for bits [1 1 1 1 1 0 1 0 0 0 0 1 1 0 1 1 0 0 0 1 0 0 0 1 1 0 1 1 1 0 0 1 0 0 0 1 1
 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 1 0 1 0]: 'PolarCodeGenerator' object has no attribute 'encode'
Encoding attempt 163:
Input Bits: [1 0 1 1 1 1 0 1 0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1
 0 0 0 0 0 0 1 1 0 1 1 1 0 0 1 1 0 1 1 0 1 1 0 1 0 0 0]
Encoding Error for bits [1 0 1 1 1 1 0 1 0 1 0 1 1 0 1 0 0 1 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1
 0 0 0 0 0 0 1 1 0 1 1 1 0 0 1 1 0 1 1 0 1 1 0 1 0 0 0]: 'PolarCodeGenerator' object has no attribute 'encode'
Encoding attempt 164:
Input Bits: [0 1 1

Traceback (most recent call last):
  File "<ipython-input-21-78ec57bde087>", line 1053, in prepare_ml_dataset_with_codewords
    raise ValueError("No valid codewords generated. Check encoding process.")
ValueError: No valid codewords generated. Check encoding process.
Traceback (most recent call last):
  File "<ipython-input-21-78ec57bde087>", line 1115, in main
    X_train, X_test, y_train, y_test = train_test_split(
                                       ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/_param_validation.py", line 216, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py", line 2850, in train_test_split
    n_samples = _num_samples(arrays[0])
                ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py", line 395, in _num_samples
    raise TypeError(message)
TypeError: Expected s